**Question:** What is ASN1?

**Answser:**
ASN stands for Abstract Syntax Notation.  
ASN1 is a standard IDL for defining data structures that can be serialized and deserialized in a cross-platform way.  
This makes it a suitable configuration file format for oai5g.  

**Sources:**
- https://en.wikipedia.org/wiki/ASN.1

**Question:** What are gNB_ID, nr_cellid and physCellId? Why are they important and how are they used?

**Answser:**
These are unique identifiers that are used in various places in the code base to refer to specific entities.

- `gNB_ID`:
    - In the RRC mac protocol
    - In the gBN application layer for NGAP messages
    - ...

- `nr_cellid`:
    - Used in L2 messages (f1ap)
    - In the RRC mac protocol
    - ...
      

`physCellId`:
    - In the RRC mac protocol
    - ...

**Sources:**
- oai5g source code

**Question:** How should you pick tracking_area_code? Is this important for our case?

**Answser:**
The Tracking Area is a logical concept of an area where a user can move around without perforing a tracking area update.  
I expect that, given our small coverage region, the tracking are will be the same for the entire region.  
Therefore, the choisc of `tracking_area_code` is arbitrary and we can leave it set to 1.  

**Sources:**
- https://www.sharetechnote.com/html/Handbook_LTE_Tracking_Area.html

**Question:** What are mcc and mnc? What should this be if you were Proximus, and what should they be for a test network?

**Answser:**
The `mcc` is the Mobile Country Code and it plays an important role in country identification and roaming functionality.

The `mnc` is the Mobile Network Code and and it is used in combination with the MCC to uniquely identify a mobile network operator (like Proximus).

Proximus specifically has an MCC/MNC tuple of 206/01,  
however for a test network it is adviced to use an MCC where the first digit is 0.  
A good tuple for a test network could be 001/001.

**Sources:**
- https://en.wikipedia.org/wiki/Mobile_country_code
- https://en.wikipedia.org/wiki/Mobile_country_code#Test_networks

**Question:** What value do you pick for dl_carrierBandwidth and why?

**Answser:**
`dl_carrierBandwidth` refers to the downlink carrier bandwidth in terms of the number of resource blocks (PRBs) allocated to the downlink transmission.

Given the 20 MHz of downlink bandwidth (see notebooks),  
the question is: how many resource blocks will fit in?  
A resource block is defined as 12 consecutive subcarriers (in the frequency domain).  
The bandwidth occupied will depend on the sub-carrier spacing (SCS).  
Given a SCS of 30 kHz (the default in the config file),  
the following equation must hold:
$$ 20 \mathrm{MHz} = x \mathrm{RB} \times 12  \mathrm{SC/RB} \times 30 \mathrm{kHz/SC} $$
Solving for x gives
$$ x = \frac{20 \times 10^3}{12 \times 30} \approx 56 RB $$

**Sources:**
- ChatGPT May 12 Version

**Question:** What would pdsch_AntennaPorts equal to 4 mean? How would this be used

**Answser:**
The `pdsch` is the Physical Downlink Shared Channel.  
It is a physical channel that delivers user data from the gNB to the UE.  
An important step in the data flow in this channel is the Antenna Port Mapping.  
More antennas improve the spatial diversity and the potential for beamforming.  

**Sources:**
- https://www.sharetechnote.com/html/5G/5G_PDSCH.html#Antenna_Port_Mapping

**Question:** Lookup the formula for EARFCN and calculate the correct values according to your parameter choices 

**Answser:**
`EARFCN` stands for E-UTRA Absolute Radio Frequency Channel Number.  
We are working with band 2 (see previous notebooks),  
which has a downlink frequency of 1960 MHz.  
For FR1, the formula becomes:

$$ N_{REF} = \frac{F_{REF}}{5 \mathrm{kHz}} $$

where $N_{REF}$ is the NR ARFCN and $F_{REF}$ is the NR frequency.  
So, plugging in 1930 MHz we get an ARFCN of 392000.  


**Sources:**
- https://www.sqimway.com/nr_band.php
- https://5g-tools.com/5g-nr-arfcn-calculator/